In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
EPOCHS = 50
BATCH_SIZE = 256

### Training Data Augmentation

In [3]:
# Training data augmentation
transform_train=transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.2,0.2,0.2)),
])

### Testing Data Normalization

In [4]:
# Normalize the test set same as training set without augmentation
transform_test=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.2,0.2,0.2))
])

### Load training and testing datasets

In [5]:
# Load the training dataset
trainset=torchvision.datasets.CIFAR10(
        root='./data',train=True,download=True,transform=transform_train)

trainloader=torch.utils.data.DataLoader(trainset,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)


Files already downloaded and verified


In [6]:
# Load the testing dataset
testset=torchvision.datasets.CIFAR10(
    root='./data',train=False,download=True,transform=transform_test)
testloader=torch.utils.data.DataLoader(testset,batch_size=BATCH_SIZE,shuffle=False,num_workers=4)

Files already downloaded and verified


In [7]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Define a Convulution Neural Network

In [8]:
import torch
import torch.nn as nn
LR = 0.005

class Net(torch.nn.Module):
    """Some Information about CNN"""

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),  
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        self.conv2 = torch.nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),  
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  
            nn.Dropout(p=0.05,inplace=False)
        )
        self.conv3 = torch.nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),  
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )

        self.conv4 = torch.nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),  
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(p=0.05,inplace=False)
        )

        self.conv5 = torch.nn.Sequential(
            nn.Conv2d(256, 256, 3, padding=1),  
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  
            nn.Dropout(p=0.05,inplace=False)
            
        )

        self.gap = nn.AvgPool2d(4,4)
        self.fc = nn.Linear(256, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.gap(x)
        x = x.view(-1, 256)
        x = self.fc(x)
        return x

net = Net()
net.cuda()


Net(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.05, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3)

### Loss Function and Optimizer

In [9]:
# Define a Loss Function and Optimizer
# I choose to use a Classification Cross-Entropy loss and Adam optimizer

import torch.optim as optim

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(),lr=0.001)

### Training Function

In [10]:
# Tain funcion

def train(model,criterion,optimizer,trainloader,epochs,log_interval=50):
    print('-------- Training job starts ------')
    for epoch in range(epochs):
        running_loss=0.0
        for step,(batch_x,batch_y) in enumerate(trainloader):
            batch_x,batch_y=batch_x.cuda(),batch_y.cuda()
            output=model(batch_x)
            
            optimizer.zero_grad()
            loss=criterion(output,batch_y)
            loss.backward()
            optimizer.step()
            
            running_loss+=loss.item()
            if step % log_interval == (log_interval-1):
                print('[%d, %5d] loss: %.4f' %
                      (epoch + 1, step + 1, running_loss / log_interval))
                running_loss = 0.0
    print('----- Train Finished -----')

### Testing Function

In [11]:
# Test function
def test(model, testloader):
    print('------ Test Start -----')

    correct = 0
    total = 0

    with torch.no_grad():
        for test_x, test_y in testloader:
            images, labels = test_x.cuda(), test_y.cuda()
            output = model(images)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Accuracy of the model is: %.4f %%' % accuracy)
    return accuracy


### Training 

In [12]:
train(net, criterion, optimizer, trainloader, epochs=EPOCHS)


-------- Training job starts ------
[1,    50] loss: 1.6755
[1,   100] loss: 1.3436
[1,   150] loss: 1.2152
[2,    50] loss: 1.0463
[2,   100] loss: 0.9687
[2,   150] loss: 0.9411
[3,    50] loss: 0.8471
[3,   100] loss: 0.8272
[3,   150] loss: 0.8120
[4,    50] loss: 0.7347
[4,   100] loss: 0.7353
[4,   150] loss: 0.6796
[5,    50] loss: 0.6517
[5,   100] loss: 0.6724
[5,   150] loss: 0.6348
[6,    50] loss: 0.6004
[6,   100] loss: 0.5966
[6,   150] loss: 0.5867
[7,    50] loss: 0.5603
[7,   100] loss: 0.5492
[7,   150] loss: 0.5389
[8,    50] loss: 0.5048
[8,   100] loss: 0.5027
[8,   150] loss: 0.5122
[9,    50] loss: 0.4802
[9,   100] loss: 0.4650
[9,   150] loss: 0.4792
[10,    50] loss: 0.4513
[10,   100] loss: 0.4431
[10,   150] loss: 0.4592
[11,    50] loss: 0.4185
[11,   100] loss: 0.4358
[11,   150] loss: 0.4361
[12,    50] loss: 0.4083
[12,   100] loss: 0.4060
[12,   150] loss: 0.4108
[13,    50] loss: 0.3863
[13,   100] loss: 0.3757
[13,   150] loss: 0.3891
[14,    50] loss

### Testing

In [13]:
test(net, testloader)

------ Test Start -----
Accuracy of the model is: 85.8300 %


85.83